In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchinfo import summary
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage import io
import pandas as pd
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file_path, root_dir, transform = None):
        # super(CustomDataset, self).__init__()
        self.data = pd.read_csv(csv_file_path)
        self.transform = transform
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.data.iloc[index, 0])
        image = io.imread(img_path)
        label = torch.tensor(self.data.iloc[index, 1])

        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [ ]:
csv_file_path = "<path_to_csv_file>"
root_dir = "<root_dir>"
batch_size = 64
learning_rate = 0.001
in_channels = 3
num_classes = 2
num_epochs = 10

data = CustomDataset(csv_file_path = csv_file_path, root_dir = root_dir, transform = transforms.ToTensor())
train_dataset, test_dataset = random_split(data, [20000, 5000])
train_loader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(test_dataset, shuffle = True, batch_size = batch_size)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 32, kernel_size = 3, stride = 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.aap = nn.AdaptiveAvgPool2d()
        self.dense1 = nn.Linear(64, 32)
        self.drop1 = nn.Dropout(0.2)
        self.dense2 = nn.Linear(32, 16)
        self.drop2 = nn.Dropout(0.2)
        self.dense3 = nn.Linear(16, 2)
        self.relu = nn.ReLU()

    def forward(self, input_tensor):
        o1 = self.relu(self.bn1(self.conv1(input_tensor)))
        o2 = self.relu(self.bn2(self.conv2(o1)))
        aap = self.aap(o2)
        dense1 = self.drop1(self.relu(self.dense1(aap)))
        dense2 = self.drop2(self.relu(self.dense2(dense1)))
        output = self.dense3(dense2)

        return output

model = SimpleCNN(in_channels, num_classes)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

print(loss_fn)
print()
print(optimizer)

In [ ]:
# Training Loop
total_loss = 0
correct = 0
losses = []
accuracies = []

model.train()
for epoch in tqdm(range(num_epochs)):
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss_batch = loss_fn(y_pred, y)

        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()

        total_loss += loss_batch.item()
        correct += (y_pred.argmax(1) == y).sum().item()
    
    losses.append(total_loss/len(train_loader.dataset))
    accuracies.append(correct/len(train_loader.dataset))

    print(f"Epoch: {epoch} | Loss: {total_loss/len(train_loader.dataset):.3f} | Accuracy: {(correct/len(train_loader.dataset)*100):.3f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5))
ax1.plot(range(num_epochs), losses)
ax1.set_title("Loss Trend")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss")

ax2.plot(range(num_epochs), accuracies)
ax2.set_title("Accuracy Trend")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Accuracy")

plt.tight_layout()
plt.show()

In [ ]:
# Testing Loop
total_loss = 0
correct = 0

model.eval()
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss_batch = loss_fn(y_pred, y)

        total_loss += loss_batch.item()
        correct += (y_pred.argmax(1) == y).sum().item()

print(f"Loss: {total_loss/len(test_loader.dataset):.3f} | Accuracy: {(correct/len(test_loader.dataset) * 100):.3f}")